# Simple example


## Data download

In [25]:
# Download dependencies
!pip install gdown

!pip install lightgbm
!pip install xgboost
!pip install catboost==1.1.1


In [2]:
# Download both interpro and rast annotations for genomes for this to work. Stored in data/ Both files are in .gitignore
import os



# Check and download allgenomes.RAST.txt.ps.tsv file
if not os.path.exists("data/allgenomes.RAST.txt.ps.tsv"):
    print("Downloading File data/allgenomes.RAST.txt.ps.tsv")
    !gdown --no-check-certificate 15py4qVFmLiHF-a341e_YX1w3HgEDwbDz -O data/allgenomes.RAST.txt.ps.tsv
else:
    print("File data/allgenomes.RAST.txt.ps.tsv already exists")

# Check and download allgenomes.interpro.txt.ps.tsv file
if not os.path.exists("data/allgenomes.interpro.txt.ps.tsv"):
    print("Downloading File data/allgenomes.interpro.txt.ps.tsv")
    !gdown --no-check-certificate 1PuDgDnWd5qumI0hkIKNYrTqICnh-_WlS -O data/allgenomes.interpro.txt.ps.tsv
else:
    print("File data/allgenomes.interpro.txt.ps.tsv already exists")

File data/allgenomes.RAST.txt.ps.tsv already exists
File data/allgenomes.interpro.txt.ps.tsv already exists


In [3]:
ls data

SSO_dictionary.json                   gram_stain_bacdive.tsv
allgenomes.RAST.txt.ps.tsv            metabolic_phenotype_data_bacdive.tsv
allgenomes.interpro.txt.ps.tsv


## Load genomic feature data

In [6]:
import pandas as pd

#Load genomic features 
#Load rast features
rast_df = pd.read_csv("data/allgenomes.RAST.txt.ps.tsv", sep = "\t", index_col=0)

#Load interpro features
interpro_df = pd.read_csv("data/allgenomes.interpro.txt.ps.tsv", sep = "\t", index_col=0)




## A simple example with gram_stain data

In [21]:
# Run a simple example
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

import pandas as pd



#Load phenotype data with gram stain data (only gram stain phenotype here) and create dataset used

phenotype_gram_stain_df = pd.read_csv("data/gram_stain_bacdive.tsv", sep = "\t", index_col=0)
phenotype = "gram_stain"
phenotype_df = phenotype_gram_stain_df[phenotype].dropna()

#Merged genotype and phenotype data and extract X and y

merged_df = rast_df.merge(phenotype_df, left_index=True, right_index=True, how='inner')
X = merged_df.drop(columns=[phenotype])
y = merged_df[phenotype]


# print some info

print ("Phenotype: " + phenotype)
print ("Feature used: Rast")

num_genome, num_features = X.shape
print ("Numer of genomes:" + str(num_genome) + "\n" + "Number of genomic features :" + str(num_features)) 
print ("Shape of y:" + str(y.shape))





# Run random forest classifier

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=112)
rfc = RandomForestClassifier(n_estimators=1000)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
confusion_matrix_info = confusion_matrix(y_test, y_pred)


print ("Accuracy = " + str(accuracy))
print ("precision = " + str(precision))
print ("Recall = " + str(recall))
print ("F1-score = " + str(f1))
print ("Balanced accuracy = " + str(balanced_accuracy))
       
    
print ("Confusion matrix:")
print (confusion_matrix_info)
       








Phenotype: gram_stain
Feature used: Rast
Numer of genomes:3531
Number of genomic features :6997
Shape of y:(3531,)
Accuracy = 0.9646393210749646
precision = 0.9658944260240436
Recall = 0.9646393210749646
F1-score = 0.9648157949771774
Balanced accuracy = 0.9679094313925775
Confusion matrix:
[[420  20]
 [  5 262]]


## More complicated example: Run multiple classifiers together, get top features and show html report



### Define functions

In [32]:
import uuid
def generate_html_table(df: pd.DataFrame):
        """Display a pandas.DataFrame as jQuery DataTables"""

        # Generate random container name
        id_container = uuid.uuid1()
        output = """
    <div id="datatable-container-{id_container}">
      <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.7.0/jquery.min.js"></script>
      <script type="text/javascript" src="https://cdn.datatables.net/1.13.5/js/jquery.dataTables.min.js"></script>
      <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.13.5/css/jquery.dataTables.min.css"/>
      <script type="text/javascript">
        $(document).ready( function () {{
            $('#BGCtable').DataTable();
        }});
      </script>
      <!-- Insert table below -->
      {table}
    </div>
        """.format(
            id_container=id_container,
            table=df.to_html(
                index=False,
                table_id="classification",
                classes="display"
            ),
        )
        return output
    

def get_classifier_report(X, y):

    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
    import xgboost as xgb
    import lightgbm as lgb
    import catboost as cb
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score
    from sklearn.feature_selection import SelectKBest, chi2
    import pandas as pd



    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Dictionary to store the classifiers and their parameters
    classifiers = {

        'Decision Tree': {
            'model': DecisionTreeClassifier(),
            'params': {}
        },
        'Random Forest': {
            'model': RandomForestClassifier(),
            'params': {'n_estimators': 1000, 'max_depth': None, 'random_state': 100}
        },
        'XGBoost': {
            'model': xgb.XGBClassifier(),
            'params': {'n_estimators': 1000, 'max_depth': 3, 'learning_rate': 0.1, 'random_state': 100}
        },

        'CatBoost': {
            'model': cb.CatBoostClassifier(),
            'params': {'iterations': 1000, 'depth': 6, 'learning_rate': 0.1, 'random_state': 42, 'verbose': False}
        },
        'AdaBoost': {
            'model': AdaBoostClassifier(),
            'params': {'n_estimators': 1000, 'random_state': 100}
        }
    }

    html_table_rows = []



    # Train and evaluate each classifier
    for clf_name, clf_data in classifiers.items():
        print ("Running " + clf_name)
        model = clf_data['model']
        params = clf_data['params']

        # Train the classifier
        model.set_params(**params)
        model.fit(X_train, y_train)

        # Make predictions on the test set
        y_pred = model.predict(X_test)



        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

        # Extract top features (importances or coefficients)
        try:
            feature_importances = dict(zip(X.columns, model.feature_importances_))
        except:
            feature_importances = {}

        # Sort and select the top 10 features
        top_features = dict(sorted(feature_importances.items(), key=lambda x: x[1], reverse=True)[:20])

        # Format the top features for prettier display
        formatted_top_features = ', '.join([f"{feat} ({importance:.2f})" for feat, importance in top_features.items()])

        # Append row to the HTML table
        html_table_rows.append([clf_name, accuracy, precision, recall, f1, balanced_accuracy, formatted_top_features])

    # Create a DataFrame to display the results
    headers = ["Model", "Accuracy", "Precision", "Recall", "F1-score", "Balanced Accuracy", "Top Features"]
    report_df = pd.DataFrame(html_table_rows, columns=headers)
    return report_df







### Run multiple classifiers on gram stain data with RAST

In [33]:
from IPython.display import display, HTML

#Load phenotype data with gram stain data (only gram stain phenotype here)
phenotype_gram_stain_df = pd.read_csv("data/gram_stain_bacdive.tsv", sep = "\t", index_col=0)

phenotype = "gram_stain"
phenotype_df = phenotype_gram_stain_df[phenotype].dropna()
merged_df = rast_df.merge(phenotype_df, left_index=True, right_index=True, how='inner')
X = merged_df.drop(columns=[phenotype])
y = merged_df[phenotype]

# Get classifier report
report_df = get_classifier_report(X,y)

report_html_content = generate_html_table (report_df)

HTML(report_html_content)


Running Decision Tree
Running Random Forest
Running XGBoost
Running CatBoost
Running AdaBoost


Model,Accuracy,Precision,Recall,F1-score,Balanced Accuracy,Top Features
Decision Tree,0.937765,0.937581,0.937765,0.937370,0.924944,"SSO:000036573 (0.72), SSO:000024182 (0.10), SSO:000001995 (0.02), SSO:000043994 (0.01), SSO:000012980 (0.01), SSO:000020168 (0.01), SSO:000004595 (0.00), SSO:000043198 (0.00), SSO:000021874 (0.00), SSO:000042298 (0.00), SSO:000017481 (0.00), SSO:000019394 (0.00), SSO:000031056 (0.00), SSO:000020170 (0.00), SSO:000012708 (0.00), SSO:000022725 (0.00), SSO:000001091 (0.00), SSO:000019142 (0.00), SSO:000016805 (0.00), SSO:000016902 (0.00)"
Random Forest,0.973126,0.974158,0.973126,0.973287,0.976597,"SSO:000007937 (0.03), SSO:000008104 (0.02), SSO:000006816 (0.02), SSO:000004460 (0.02), SSO:000036573 (0.02), SSO:000008604 (0.02), SSO:000043994 (0.01), SSO:000020865 (0.01), SSO:000000281 (0.01), SSO:000002241 (0.01), SSO:000009323 (0.01), SSO:000002230 (0.01), SSO:000004958 (0.01), SSO:000008598 (0.01), SSO:000008953 (0.01), SSO:000019343 (0.01), SSO:000021918 (0.01), SSO:000023921 (0.01), SSO:000004949 (0.01), SSO:000035277 (0.01)"
XGBoost,0.964639,0.965278,0.964639,0.964789,0.965192,"SSO:000036573 (0.40), SSO:000024182 (0.03), SSO:000006816 (0.03), SSO:000043994 (0.02), SSO:000002793 (0.02), SSO:000001995 (0.02), SSO:000008306 (0.02), SSO:000001301 (0.01), SSO:000042832 (0.01), SSO:000017976 (0.01), SSO:000008604 (0.01), SSO:000008104 (0.01), SSO:000008630 (0.01), SSO:000012980 (0.01), SSO:000008066 (0.01), SSO:000008135 (0.01), SSO:000018456 (0.01), SSO:000012951 (0.01), SSO:000019152 (0.01), SSO:000001853 (0.01)"
CatBoost,0.966054,0.967032,0.966054,0.966243,0.968248,"SSO:000036573 (28.02), SSO:000004460 (5.10), SSO:000043994 (4.39), SSO:000008104 (3.37), SSO:000006816 (3.00), SSO:000007937 (2.09), SSO:000024182 (1.98), SSO:000008604 (1.66), SSO:000001301 (1.48), SSO:000008105 (1.34), SSO:000002076 (1.21), SSO:000013395 (1.14), SSO:000004190 (1.05), SSO:000012160 (0.98), SSO:000002241 (0.89), SSO:000012811 (0.86), SSO:000000563 (0.81), SSO:000008306 (0.56), SSO:000006040 (0.53), SSO:000000281 (0.53)"
AdaBoost,0.963225,0.963225,0.963225,0.963225,0.959162,"SSO:000043092 (0.01), SSO:000008104 (0.01), SSO:000016911 (0.01), SSO:000018986 (0.01), SSO:000029838 (0.01), SSO:000012182 (0.01), SSO:000042756 (0.01), SSO:000001768 (0.01), SSO:000024895 (0.01), SSO:000001968 (0.01), SSO:000044120 (0.01), SSO:000008604 (0.01), SSO:000020269 (0.01), SSO:000012160 (0.01), SSO:000024618 (0.01), SSO:000020490 (0.01), SSO:000010833 (0.01), SSO:000025091 (0.01), SSO:000004100 (0.01), SSO:000005483 (0.01)"


In [35]:
### Find feature details (Only for RAST features).. Not FOR INTERPRO

import json
def find_rast_details(report_string):
    with open ("data/SSO_dictionary.json") as sf:
           sso_terms = json.load(sf)['term_hash']
    sso_dict = dict()
    for term in sso_terms:
        data = sso_terms[term]
        sso_dict[data['id']] = data['name']
        
    report_data = report_string.split(",")
    for rast_score in report_data:
        rast_score = rast_score.lstrip()
        rast_id = rast_score.split(" ")[0]
        
        print (rast_score + "\t" + sso_dict[rast_id])
        
        
report_string = "SSO:000036573 (0.71), SSO:000024182 (0.10), SSO:000001301 (0.02), SSO:000012980 (0.01), SSO:000031056 (0.01), SSO:000020884 (0.01), SSO:000043940 (0.01), SSO:000002109 (0.01), SSO:000004554 (0.00), SSO:000025018 (0.00), SSO:000021037 (0.00), SSO:000000212 (0.00), SSO:000016804 (0.00), SSO:000044352 (0.00), SSO:000020878 (0.00), SSO:000012160 (0.00), SSO:000021268 (0.00), SSO:000017255 (0.00), SSO:000033511 (0.00), SSO:000017940 (0.00)"
find_rast_details(report_string)


SSO:000036573 (0.71)	LPS-assembly protein LptD
SSO:000024182 (0.10)	Ribosome maturation factor rimM
SSO:000001301 (0.02)	CDP-diacylglycerol--glycerol-3-phosphate 3-phosphatidyltransferase (EC 2.7.8.5)
SSO:000012980 (0.01)	DUF2225 domain-containing protein
SSO:000031056 (0.01)	metalloenzyme domain protein
SSO:000020884 (0.01)	Outer membrane lipoprotein-sorting protein
SSO:000043940 (0.01)	dihydrolipoyllysine-residue acetyltransferase (EC 2.3.1.12)
SSO:000002109 (0.01)	DNA repair protein RecN
SSO:000004554 (0.00)	Low-specificity L-threonine aldolase (EC 4.1.2.48)
SSO:000025018 (0.00)	Spore coat protein CotH
SSO:000021037 (0.00)	PAS domain-containing protein
SSO:000000212 (0.00)	2-phosphosulfolactate phosphatase (EC 3.1.3.71 )
SSO:000016804 (0.00)	Flagellar motor protein
SSO:000044352 (0.00)	formate C-acetyltransferase (EC 2.3.1.54)
SSO:000020878 (0.00)	Outer membrane lipoprotein carrier protein LolA
SSO:000012160 (0.00)	Chemotaxis response regulator protein-glutamate methylesterase (EC 3

### Run multiple classifiers on gram stain data with Interpro


In [34]:
# Run multiple classifiers on gram stain data with Interpro
from IPython.display import display, HTML

#Load phenotype data with gram stain data (only gram stain phenotype here)
phenotype_gram_stain_df = pd.read_csv("data/gram_stain_bacdive.tsv", sep = "\t", index_col=0)

phenotype = "gram_stain"
phenotype_df = phenotype_gram_stain_df[phenotype].dropna()
merged_df = interpro_df.merge(phenotype_df, left_index=True, right_index=True, how='inner')
X = merged_df.drop(columns=[phenotype])
y = merged_df[phenotype]

# Get classifier report
report_df = get_classifier_report(X,y)

report_html_content = generate_html_table (report_df)

HTML(report_html_content)


Running Decision Tree
Running Random Forest
Running XGBoost
Running CatBoost
Running AdaBoost


Model,Accuracy,Precision,Recall,F1-score,Balanced Accuracy,Top Features
Decision Tree,0.940711,0.940548,0.940711,0.940613,0.930268,"IPR000184 (0.78), IPR002543 (0.05), IPR043723 (0.02), IPR004550 (0.01), IPR007835 (0.01), IPR000109 (0.01), IPR015262 (0.01), IPR008307 (0.01), IPR009293 (0.01), IPR003561 (0.01), IPR008767 (0.01), IPR019539 (0.01), IPR008567 (0.00), IPR013570 (0.00), IPR003468 (0.00), IPR005064 (0.00), IPR036473 (0.00), IPR001765 (0.00), IPR005235 (0.00), IPR017502 (0.00)"
Random Forest,0.974308,0.975346,0.974308,0.974485,0.977839,"IPR039518 (0.01), IPR003400 (0.01), IPR005653 (0.01), IPR039910 (0.01), IPR023052 (0.01), IPR010827 (0.01), IPR004474 (0.01), IPR015349 (0.01), IPR036942 (0.01), IPR030921 (0.01), IPR037066 (0.01), IPR005495 (0.01), IPR028053 (0.01), IPR012910 (0.01), IPR023707 (0.01), IPR027434 (0.01), IPR039426 (0.01), IPR038594 (0.01), IPR039565 (0.01), IPR004613 (0.01)"
XGBoost,0.964427,0.965063,0.964427,0.964594,0.964040,"IPR000184 (0.15), IPR005495 (0.12), IPR004474 (0.06), IPR002543 (0.04), IPR018478 (0.03), IPR042214 (0.02), IPR004570 (0.02), IPR023473 (0.02), IPR043723 (0.01), IPR020988 (0.01), IPR020059 (0.01), IPR035924 (0.01), IPR002043 (0.01), IPR025097 (0.01), IPR022641 (0.01), IPR046786 (0.01), IPR010969 (0.01), IPR036105 (0.01), IPR034762 (0.01), IPR003344 (0.01)"
CatBoost,0.974308,0.975004,0.974308,0.974448,0.976206,"IPR005495 (14.29), IPR003400 (9.55), IPR004474 (4.25), IPR039910 (3.32), IPR036346 (2.98), IPR003802 (2.60), IPR000184 (1.55), IPR015870 (1.33), IPR017501 (0.95), IPR006412 (0.95), IPR003423 (0.89), IPR006256 (0.75), IPR004800 (0.67), IPR023087 (0.65), IPR029050 (0.63), IPR013564 (0.63), IPR004570 (0.63), IPR017689 (0.60), IPR027463 (0.60), IPR018958 (0.56)"
AdaBoost,0.962451,0.962707,0.962451,0.962541,0.959320,"IPR046674 (0.01), IPR024726 (0.01), IPR000428 (0.01), IPR037274 (0.01), IPR037483 (0.01), IPR040074 (0.01), IPR000330 (0.01), IPR029095 (0.01), IPR011304 (0.01), IPR024728 (0.01), IPR045926 (0.01), IPR015330 (0.01), IPR025984 (0.01), IPR025357 (0.01), IPR014084 (0.01), IPR015919 (0.01), IPR045528 (0.01), IPR008207 (0.01), IPR007168 (0.01), IPR038901 (0.01)"


In [41]:
### Find feature details (Only for RAST features).. Not FOR INTERPRO

import json
def find_interpro_details(report_string):
    with open ("data/Interpro.list") as ipro:
           interpro_terms = ipro.readlines()
            
    interpro_dict = dict()
    
    for line in interpro_terms:
        line = line.rstrip()
        id =  line.split("\t")[0]
        name = line.split("\t")[2]
        interpro_dict[id] = name
        
        
    report_data = report_string.split(",")
    for interpro_score in report_data:
        interpro_score = interpro_score.lstrip()
        interpro_id = interpro_score.split(" ")[0]
        
        print (interpro_score + "\t" + interpro_dict[interpro_id])
        
       
report_string = "IPR000184 (0.78), IPR002543 (0.05), IPR043723 (0.02), IPR004550 (0.01), IPR007835 (0.01), IPR000109 (0.01), IPR015262 (0.01), IPR008307 (0.01), IPR009293 (0.01), IPR003561 (0.01), IPR008767 (0.01), IPR019539 (0.01), IPR008567 (0.00), IPR013570 (0.00), IPR003468 (0.00), IPR005064 (0.00), IPR036473 (0.00), IPR001765 (0.00), IPR005235 (0.00), IPR017502 (0.00)"
find_interpro_details(report_string)


IPR000184 (0.78)	Bacterial surface antigen (D15)
IPR002543 (0.05)	FtsK domain
IPR043723 (0.02)	Domain of unknown function DUF5665
IPR004550 (0.01)	L-asparaginase, type II
IPR007835 (0.01)	MOFRL domain
IPR000109 (0.01)	Proton-dependent oligopeptide transporter family
IPR015262 (0.01)	tRNA(Ile)-lysidine synthase, substrate-binding domain
IPR008307 (0.01)	Uncharacterised conserved protein UCP018957
IPR009293 (0.01)	Uncharacterised protein family UPF0478
IPR003561 (0.01)	Mutator MutT
IPR008767 (0.01)	Bacteriophage SPP1, head-tail adaptor
IPR019539 (0.01)	Galactokinase, N-terminal domain
IPR008567 (0.00)	Beta-keto acid cleavage enzymes
IPR013570 (0.00)	Transcription regulator YsiA, C-terminal
IPR003468 (0.00)	Cytochrome c oxidase, monohaem subunit/FixO
IPR005064 (0.00)	Bordetella uptake gene
IPR036473 (0.00)	Molybdopterin cofactor biosynthesis MoaD-related, C-terminal domain superfamily
IPR001765 (0.00)	Carbonic anhydrase
IPR005235 (0.00)	Metallophosphoesterase, YmdB-like
IPR017502 (0.00)	S